<h1 align="center"> <span style="font-size: 20px;"> Proyecto Final Grupo Nº 7 </span> <br>
    <align="center"> <span style="font-size: 15px;">  Proyecto Yelp & Google Maps - Reviewa y Recomendaciones</span><br>         
<align="center"> <span style="font-size: 15px;">@utores:  Javier Castro, Luca Ramallo, Luis Ramírez, Lesmen García.</span> <br>
  <a href="https://github.com/lesmengp/Proyecto-Final-Grupo-07.git">GitHub: <span style="font-size: 20px;">Proyecto Yelp & Google Maps</span></a> 
</h1>

### Librerías a Utilizar:

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import os
import io
import json
import gcsfs

In [3]:
import nltk
nltk.download('vader_lexicon')   # VADER es una Biblioteca que ya contirnr algunas reglas para el Analisis de Sentimientos. Asigna puntuaciones

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/jupyter/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [4]:
from nltk.sentiment import SentimentIntensityAnalyzer  # Nos da una probabilidad si el mensaje es negativo, neutro o positivo. 

In [5]:
sia = SentimentIntensityAnalyzer() # Se instancia el modelo

In [5]:
### Cargando el ETL del Dataset 'yelp_bussines'
# Ruta completa en Cloud Storage
ruta_eda_datalike = "gs://gmy/eda/yelp_reviews.parquet"

# Lee el archivo Parquet directamente en un DataFrame de pandas
yelp_reviews = pd.read_parquet(ruta_eda_datalike, storage_options={"project": "Proyecto Final - Henry"})
yelp_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2742210 entries, 0 to 2742209
Data columns (total 8 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   review_id    object
 1   user_id      object
 2   business_id  object
 3   stars        int64 
 4   useful       int64 
 5   cool         int64 
 6   text         object
 7   date         object
dtypes: int64(3), object(5)
memory usage: 167.4+ MB


In [12]:
def AnalizarSentimiento(texto):
    sia = SentimentIntensityAnalyzer() # Se instancia el modelo
    sentimiento = sia.polarity_scores(texto)["compound"]
    
    if sentimiento > 0.1:
        return "Sentimiento Positivo"
    else:
        if sentimiento < 0:
            return "Sentimiento Negativo"
        else:
            return "Sentimiento Neutro"

In [7]:
f1 = "I love this place"
f2 = "I hate this restaurants"
f3 = "I have nothing to say"
f4= "La comida de este restaurante es buena"
f5 = "Volvería a comer a este restaurante"

In [18]:
f = f1 # Seleccionando Frase para analizar
resultadoSentimiento = sia.polarity_scores(f)
print(resultadoSentimiento)
print("Resultado: ", AnalizarSentimiento(f))

{'neg': 0.0, 'neu': 0.323, 'pos': 0.677, 'compound': 0.6369}
Resultado:  Sentimiento Positivo


In [19]:
f = f2 # Seleccionando Frase para analizar
resultadoSentimiento = sia.polarity_scores(f)
print(resultadoSentimiento)
print("Resultado: ", AnalizarSentimiento(f))

{'neg': 0.649, 'neu': 0.351, 'pos': 0.0, 'compound': -0.5719}
Resultado:  Sentimiento Negativo


In [20]:
f = f3 # Seleccionando Frase para analizar
resultadoSentimiento = sia.polarity_scores(f)
print(resultadoSentimiento)
print("Resultado: ", AnalizarSentimiento(f))

{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
Resultado:  Sentimiento Neutro


### El siguiente texto deberia arrojar un sentimiento positivo y no lo hace.

In [24]:
f4= "La comida de este restaurante es buena"     # Deberia clasificarla como POSITIVO
# Esta frase representa una opinión explícita, el párrafo indica que es una frase positiva.
f = f4 # Seleccionando Frase para analizar
resultadoSentimiento = sia.polarity_scores(f)
print(resultadoSentimiento)
print("Resultado: ", AnalizarSentimiento(f))

{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
Resultado:  Sentimiento Neutro


### El siguiente texto deberia arrojar un sentimiento positivo y no lo hace.

In [26]:
f5 = "Volvería a comer a este restaurante"    # Deberia clasificarla como POSITIVO
#  Esta frase representa una opinión implícita, es decir, en la frase no se ve ninguna palabra que indique
#  positividad, pero la connotación de la misma es claramente positiva
f = f5 # Seleccionando Frase para analizar
resultadoSentimiento = sia.polarity_scores(f)
print(resultadoSentimiento)
print("Resultado: ", AnalizarSentimiento(f))

{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
Resultado:  Sentimiento Neutro


### En resumen este modelo de NLTK con VADE no clasifica bien!!!!

In [27]:
f6 = "Me ha gustado mucho este restaurante"    # Deberia clasificarla como POSITIVO
#  Esta frase representa una opinión implícita, es decir, en la frase no se ve ninguna palabra que indique
#  positividad, pero la connotación de la misma es claramente positiva
f = f6 # Seleccionando Frase para analizar
resultadoSentimiento = sia.polarity_scores(f)
print(resultadoSentimiento)
print("Resultado: ", AnalizarSentimiento(f))

{'neg': 0.0, 'neu': 0.676, 'pos': 0.324, 'compound': 0.34}
Resultado:  Sentimiento Positivo


In [11]:
yelp_reviews['text']

0          If you decide to eat here, just be aware it is...
1          I was really between 3 and 4 stars for this on...
2          My boyfriend and I tried this deli for the fir...
3          Amazing biscuits and (fill in the blank). Grea...
4          The cafe was extremely cute. We came at 8am an...
                                 ...                        
2742205    I've stayed here before and had no issues with...
2742206    This was my first time here and omg it was ama...
2742207    The service was great the food was fantastic t...
2742208    Great food, great people, local bar with stron...
2742209    I like this place! Staff are always friendly a...
Name: text, Length: 2742210, dtype: object

In [12]:
yelp_reviews.shape

(2742210, 8)

In [ ]:
### Este Funcion empiza pero no finaliza. Debe ser por la cantidad de Registros
#    yelp_reviews['Sentimiento'] = yelp_reviews['text'].apply(AnalizarSentimiento)